In [1]:
import json
import pandas as pd
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [64]:
metadata = []
with open('scraped_metadata3/MasterMetadata3.txt','r') as f:
    for line in f:
        metadata.append(json.loads(line.strip()))
for item in metadata:
    item['categories'] = ', '.join(item['categories'].keys())
df = pd.DataFrame.from_dict(metadata)
#df.to_csv('MasterMetadata2.csv',encoding='utf-8')

In [ ]:
#df[df['city']=='San Francisco'][['BizName','latitude','longitude']].to_csv('SanFranciscoLocations.csv',encoding='utf-8',index=False)

In [117]:
simscores = []
with open('scraped_reviews/ElFarolitoSimilarities.csv','r') as f:
    for line in f:
        simscores.append(line)

In [118]:
simsdf = pd.DataFrame.from_csv('scraped_reviews/ElFarolitoSimilarities.csv')

In [119]:
dfm = simsdf.merge(df,on='BizName')

In [120]:
dfm

,BizName,Score,Color,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Ages Allowed,Alcohol,Ambience,Attire,...,Wi-Fi,categories,city,href,latitude,longitude,neighborhood,state,street address,zipcode
0,El Farolito,0.000000,#00f,NaN,NaN,Yes,NaN,Beer & Wine Only,Casual,Casual,...,No,Mexican,San Francisco,/biz/el-farolito-san-francisco,37.721131,-122.437161,Excelsior,CA,4817 Mission St,94112
1,El Farolito,0.000000,#00f,No,NaN,No,NaN,Beer & Wine Only,Divey,Casual,...,No,Mexican,San Francisco,/biz/el-farolito-san-francisco-2,37.752654,-122.418192,Mission,CA,2779 Mission St,94110
2,The Sentinel,0.666667,#fff,No,NaN,Yes,NaN,No,Casual,Casual,...,No,Sandwiches,San Francisco,/biz/the-sentinel-san-francisco,37.788298,-122.401372,"Financial District, SoMa",CA,37 New Montgomery St,94105
3,Garaje,0.692913,#fcfcfc,No,NaN,No,NaN,Beer & Wine Only,Casual,Casual,...,No,"Burgers, Gastropubs, Mexican",San Francisco,/biz/garaje-san-francisco,37.781663,-122.396104,SoMa,CA,475 3rd St,94107
4,Good Mong Kok Bakery,0.699029,#fbfbfb,No,NaN,No,NaN,No,Casual,Casual,...,No,"Bakeries, Dim Sum, Cantonese",San Francisco,/biz/good-mong-kok-bakery-san-francisco,37.795509,-122.408236,Chinatown,CA,1039 Stockton St,94108
5,Saigon Sandwich,0.720000,#f9f9f9,No,NaN,No,NaN,No,Casual,Casual,...,No,"Sandwiches, Vietnamese",San Francisco,/biz/saigon-sandwich-san-francisco,37.783060,-122.417337,Tenderloin,CA,560 Larkin St,94102
6,Kitchen Story,0.724138,#f8f8f8,No,NaN,Yes,NaN,Beer & Wine Only,Casual,Casual,...,Free,"Asian Fusion, Breakfast & Brunch, American (New)",San Francisco,/biz/kitchen-story-san-francisco,37.764355,-122.430868,Castro,CA,3499 16th St,94114
7,Top Dog,0.741497,#f7f7f7,No,NaN,No,NaN,No,Casual,Casual,...,No,Hot Dogs,Berkeley,/biz/top-dog-berkeley,37.867721,-122.257384,Telegraph Ave,CA,2534 Durant Ave,94704
8,Top Dog,0.741497,#f7f7f7,NaN,NaN,Yes,NaN,No,Casual,Casual,...,No,Hot Dogs,Oakland,/biz/top-dog-oakland-2,37.810638,-122.244342,"Lower Hills, Lakeshore",CA,3272 Lakeshore Ave,94610
9,Kokkari Estiatorio,0.752525,#f6f6f6,NaN,NaN,Yes,NaN,Full Bar,Classy,Dressy,...,No,"Greek, Mediterranean",San Francisco,/biz/kokkari-estiatorio-san-francisco,37.797036,-122.399755,Financial District,CA,200 Jackson St,94111


In [68]:
dfm[['BizName','latitude','longitude','Color','Score','categories','href']].to_csv('ElFarolitoSimilarities.csv',encoding='utf-8',index=False)

In [121]:
gjl = []
for ind, item in dfm.iterrows():
    gj = dict()
    gj['type'] = 'Feature'
    gj['geometry'] = {'type':'Point', 'coordinates':[item['longitude'],item['latitude']]}
    desc = 'Rating Differential: '+str(item['Score'])+'\nCategories: '+item['categories']
    gj['properties'] = {'title':item['BizName'], 'description' : desc, 'marker-color' : item['Color'], 
                        "marker-size": "medium"}
    gjl.append(gj)

In [104]:
gjl[0]

{'geometry': {'coordinates': [37.7211313, -122.4371614], 'type': 'Point'},
 'properties': {'description': u'Rating Differential: 0.0\nCategories: Mexican',
  'title': 'El Farolito'},
 'type': 'Feature'}

In [122]:
with open('ElFarolitoSimilarities.geojson','w') as f:
    for line in json.dumps(gjl,indent=2, separators=(',', ': ')):
        f.write(line.encode('utf-8'))

In [18]:
# Markov cluster model
from mcl.mcl_clustering import networkx_mcl

In [19]:
dir(networkx_mcl)

['__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__hash__',
 '__init__',
 '__module__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'func_closure',
 'func_code',
 'func_defaults',
 'func_dict',
 'func_doc',
 'func_globals',
 'func_name']

In [ ]:
G = <your graph>

M, clusters = networkx_mcl(G, expand_factor = <expand_factor>,
                   inflate_factor = <inflate_factor>,
                   max_loop = <max_loop>,
                   mult_factor = <mult_factor>)